# SimPy Tutorial
https://towardsdatascience.com/introduction-to-simulation-with-simpy-322606d4ba0c

https://simpy.readthedocs.io/en/latest/simpy_intro/basic_concepts.html

In [1]:
import simpy
import pandas as pd
import numpy  as np
from scipy.stats import norm
from scipy.stats import expon

In [2]:
def car(env):
     while True:
         print('Start parking at %d' % env.now)
         parking_duration = 5
         yield env.timeout(parking_duration)
         print('Start driving at %d' % env.now)
         trip_duration = 2
         yield env.timeout(trip_duration)

In [3]:
env = simpy.Environment()
env.process(car(env))

env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


In [4]:
class Car(object):
     def __init__(self, env):
         self.env = env
         # Start the run process everytime an instance is created.
         self.action = env.process(self.run())

     def run(self):
         while True:
             print('Start parking and charging at %d' % self.env.now)
             charge_duration = 5
             # We yield the process that process() returns
             # to wait for it to finish
             yield self.env.process(self.charge(charge_duration))

             # The charge process has finished and
             # we can start driving again.
             print('Start driving at %d' % self.env.now)
             trip_duration = 2
             yield self.env.timeout(trip_duration)

     def charge(self, duration):
         yield self.env.timeout(duration)

In [5]:
env = simpy.Environment()
car = Car(env)
#env.process(car(env))

env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


## SimPy: interrupting processes via interruption handling (interrupted object/class/process)

In [6]:
def driver(env, car):
     yield env.timeout(3)
     car.action.interrupt()

In [7]:
class Car(object):
     def __init__(self, env):
         self.env = env
         # Start the run process everytime an instance is created.
         self.action = env.process(self.run())

     def run(self):
         while True:
             print('Start parking and charging at %d' % self.env.now)
             charge_duration = 5
            # We may get interrupted while charging the battery
             try:
                 yield self.env.process(self.charge(charge_duration))
             except simpy.Interrupt:
                 # When we received an interrupt, we stop charging and
                 # switch to the "driving" state
                 print(f'Was interrupted at {env.now}. Hope, the battery is full enough ...')

             # The charge process has finished and
             # we can start driving again.
             print('Start driving at %d' % self.env.now)
             trip_duration = 2
             yield self.env.timeout(trip_duration)

     def charge(self, duration):
         yield self.env.timeout(duration)

In [8]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))

env.run(until=15)

Start parking and charging at 0
Was interrupted at 3. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


#### Resources
- basic resources: FIFO

In [9]:
def car(env, name, bcs, driving_time, charge_duration):
     """BCS: battery charging station"""
     # Simulate driving to the BCS
     yield env.timeout(driving_time)

     # Request one of its charging spots
     print('%s arriving at %d' % (name, env.now))
     # with statement automatically releases resources
     # otherwise release() needs to be called separately
     with bcs.request() as req:
         yield req

         # Charge the battery
         print('%s starting to charge at %s' % (name, env.now))
         yield env.timeout(charge_duration)
         print('%s leaving the bcs at %s' % (name, env.now))

In [10]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)

In [11]:
for i in range(4):
     env.process(car(env, 'Car %d' % i, bcs, i*2, 5))

In [12]:
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12


In [13]:
import random
import simpy

In [14]:
"""
Machine shop example

Covers:

- Interrupts
- Resources: PreemptiveResource

Scenario:
  A workshop has *n* identical machines. A stream of jobs (enough to
  keep the machines busy) arrives. Each machine breaks down
  periodically. Repairs are carried out by one repairman. The repairman
  has other, less important tasks to perform, too. Broken machines
  preempt theses tasks. The repairman continues them when he is done
  with the machine repair. The workshop works continuously.

"""

RANDOM_SEED = 42
PT_MEAN = 10.0         # Avg. processing time in minutes
PT_SIGMA = 2.0         # Sigma of processing time
MTTF = 300.0           # Mean time to failure in minutes
BREAK_MEAN = 1 / MTTF  # Param. for expovariate distribution
REPAIR_TIME = 30.0     # Time it takes to repair a machine in minutes
JOB_DURATION = 30.0    # Duration of other jobs in minutes
NUM_MACHINES = 10      # Number of machines in the machine shop
WEEKS = 4              # Simulation time in weeks
SIM_TIME = WEEKS * 7 * 24 * 60  # Simulation time in minutes


def time_per_part():
    """Return actual processing time for a concrete part."""
    return random.normalvariate(PT_MEAN, PT_SIGMA)


def time_to_failure():
    """Return time until next failure for a machine."""
    return random.expovariate(BREAK_MEAN)


class Machine(object):
    """A machine produces parts and my get broken every now and then.

    If it breaks, it requests a *repairman* and continues the production
    after the it is repaired.

    A machine has a *name* and a numberof *parts_made* thus far.

    """
    def __init__(self, env, name, repairman):
        self.env = env
        self.name = name
        self.parts_made = 0
        self.broken = False

        # Start "working" and "break_machine" processes for this machine.
        self.process = env.process(self.working(repairman))
        env.process(self.break_machine())

    def working(self, repairman):
        """Produce parts as long as the simulation runs.

        While making a part, the machine may break multiple times.
        Request a repairman when this happens.

        """
        while True:
            # Start making a new part
            done_in = time_per_part()
            while done_in:
                try:
                    # Working on the part
                    start = self.env.now
                    yield self.env.timeout(done_in)
                    done_in = 0  # Set to 0 to exit while loop.

                except simpy.Interrupt:
                    self.broken = True
                    done_in -= self.env.now - start  # How much time left?

                    # Request a repairman. This will preempt its "other_job".
                    with repairman.request(priority=1) as req:
                        yield req
                        yield self.env.timeout(REPAIR_TIME)

                    self.broken = False

            # Part is done.
            self.parts_made += 1

    def break_machine(self):
        """Break the machine every now and then."""
        while True:
            yield self.env.timeout(time_to_failure())
            if not self.broken:
                # Only break the machine if it is currently working.
                self.process.interrupt()


def other_jobs(env, repairman):
    """The repairman's other (unimportant) job."""
    while True:
        # Start a new job
        done_in = JOB_DURATION
        while done_in:
            # Retry the job until it is done.
            # It's priority is lower than that of machine repairs.
            with repairman.request(priority=2) as req:
                yield req
                try:
                    start = env.now
                    yield env.timeout(done_in)
                    done_in = 0
                except simpy.Interrupt:
                    done_in -= env.now - start

In [15]:
# Setup and start the simulation
print('Machine shop')
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
repairman = simpy.PreemptiveResource(env, capacity=1)
machines = [Machine(env, 'Machine %d' % i, repairman)
            for i in range(NUM_MACHINES)]
env.process(other_jobs(env, repairman))

# Execute!
env.run(until=SIM_TIME)

# Analyis/results
print('Machine shop results after %s weeks' % WEEKS)
for machine in machines:
    print('%s made %d parts.' % (machine.name, machine.parts_made))

Machine shop
Machine shop results after 4 weeks
Machine 0 made 3251 parts.
Machine 1 made 3273 parts.
Machine 2 made 3242 parts.
Machine 3 made 3343 parts.
Machine 4 made 3387 parts.
Machine 5 made 3244 parts.
Machine 6 made 3269 parts.
Machine 7 made 3185 parts.
Machine 8 made 3302 parts.
Machine 9 made 3279 parts.


In [16]:
random.normalvariate()

1.3192211945238288

## JSSP instance

In [17]:
import numpy as np
import numpy.typing as npt
import simpy

In [18]:
def gen_rnd_JSSP_inst(
    n_jobs: int,
    n_machines: int,
    seed: int = 42,
) -> tuple[int, int, int, npt.NDArray[np.uint16], npt.NDArray[np.uint16], npt.NDArray[np.uint16]]:
    """
    Generates random job shop instance with given number of jobs and machines'
    - each job on all machines
    - max processing time = 9
    
    Output:
        - n_jobs: number of jobs
        - n_machines: number of machines
        - n_tasks: number of tasks
        - mat_ProcTimes: matrix of processing times | shape=(n_jobs,n_machines)
        - mat_JobMachID: matrix of machine IDs per job starting by index 1 | shape=(n_jobs,n_machines)
        - mat_OpID: matrix of operation IDs starting by index 1 | shape=(n_jobs,n_machines)
    """
    # generate random process time matrix shape=(n_jobs, n_machines)
    np_rnd_gen = np.random.default_rng(seed=seed)
    mat_ProcTimes = np_rnd_gen.integers(1, 10, size=(n_jobs,n_machines), dtype=np.uint16)
    
    # generate randomly shuffled job machine combinations
    # machine IDs from 1 to n_machines
    temp = np.arange(0, (n_machines), step=1, dtype=np.uint16)
    temp = np.expand_dims(temp, axis=0)
    # repeat dummy line until number n_jobs is reached
    temp = np.repeat(temp, n_jobs, axis=0)
    # randomly permute the machine indices job-wise
    mat_JobMachID = np_rnd_gen.permuted(temp, axis=1)
    
    # generate operation ID matrix
    n_ops = n_jobs * n_machines
    temp2 = np.arange(0, (n_ops), step=1, dtype=np.uint16)
    mat_OpID = temp2.reshape(n_jobs, -1)
    
    return n_jobs, n_machines, n_ops, mat_ProcTimes, mat_JobMachID, mat_OpID

In [19]:
(n_jobs, n_machines, n_ops, 
 mat_ProcTimes, mat_JobMachID, mat_OpID) = gen_rnd_JSSP_inst(2,3)

In [20]:
mat_ProcTimes

array([[2, 1, 9],
       [7, 9, 6]], dtype=uint16)

In [21]:
mat_JobMachID

array([[2, 0, 1],
       [0, 1, 2]], dtype=uint16)

##### Brainstorming of ways to import machine names and IDs
*Case 1: only names are given*
- read machine names and assert IDs to them
- build data structure with name and ID bundled (maybe as property of a machine class)

*Case 2: IDs are given*
- only building data structure with name and ID bundled

*Data Structure:*
- if only mapping of two pairs in each direction (lookup ID or lookup machine name)
    - bi-directional dictionary

In [22]:
import numpy as np
import numpy.typing as npt
import simpy
from typing import TypeAlias
from collections import OrderedDict
from collections import deque
import logging
import sys
import pandas as pd
from pandas import DataFrame
from pandas import Series
from functools import lru_cache

# type aliases
SimPyEnv: TypeAlias = simpy.core.Environment
EnvID: TypeAlias = int
JobID: TypeAlias = int
OpID: TypeAlias = int
MachineID: TypeAlias = int | str
CustomID: TypeAlias = int | str
InfstructObj: TypeAlias = object # better naming in future

# forward reference, referenced before assignment
#Job: TypeAlias = 'Job'
#Dispatcher: TypeAlias = 'Dispatcher'

# logging
# IPython compatibility
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
#LOGGING_LEVEL = 'DEBUG'
logger = logging.getLogger('base')
#logger.setLevel(LOGGING_LEVEL)

**for resource objects:**
- CustomID may not be of type 'None' because the custom identifiers are the only interface to the end user (EnvID are solely handled inernally)
- add checking for uniqueness of custom identifiers necessary, else the mapping of different objects could be ambiguous
- jobs and operations can use ambiguous custom IDs --> custom IDs can still be None

- dedicated environment class with information on associated resources and jobs
- maybe add possibility of using subsystems (bundle of resources with unique identifiers)

In [23]:
class SimulationEnvironment(simpy.core.Environment):
    Dispatcher: TypeAlias = 'Dispatcher'
    
    def __init__(
        self,
        *args,
        **kwargs,
    ) -> None:
        """
        
        """
        super().__init__(*args, **kwargs)
        
        # resource data base as simple Pandas DataFrame
        self._infstruct_prop: dict[str, type] = {
            'env_id': int,
            'custom_id': object,
            'resource': object,
            'name': str,
            'res_type': str,
        }
        self._res_db: DataFrame = pd.DataFrame(columns=list(self._infstruct_prop.keys()))
        self._res_db: DataFrame = self._res_db.astype(self._infstruct_prop)
        self._res_lookup_props: set[str] = set(['env_id', 'custom_id', 'name'])
        
        # env identifiers
        self._id_counter: EnvID = 0
        self._res_custom_identifiers: set[str | int] = set()
        
        # job dispatcher
        self._dispatcher_registered: bool = False
        self._dispatcher: Dispatcher = None
        
        ############## LEGACY CODE
        ### legacy code, changed approach to tabular data structure
        """
        self.id_counter: EnvID = 0
        self.resources: dict[EnvID, object] = dict()
        self._custom_identifiers: set[CustomID] = set()
        self._custom_from_env_ids: dict[EnvID, CustomID] = dict()
        self._custom_to_env_ids: dict[CustomID, EnvID] = dict()
        """
        
    def _obtain_env_id(self) -> EnvID:
        """Simple counter function for managing environment IDs"""
        # assign id and set counter up
        env_id = self._id_counter
        self._id_counter += 1
        
        return env_id
    
    def register_dispatcher(
        self,
        dispatcher: Dispatcher,
    ) -> EnvID:
        """
        Registers a dispatcher instance for the environment. Only one instance per environment is allowed.
        returns: EnvID for the dispatcher instance
        """
        # obtain env_id
        env_id = self._obtain_env_id()
        
        if not self._dispatcher_registered:
            self._dispatcher = dispatcher
            self._dispatcher_registered = True
            logger.info(f"Successfully registered dispatcher with EnvID {env_id}")
        else:
            raise AssertionError("There is already a registered dispatcher instance \
                                 Only one instance per environement is allowed.")
        
        return env_id
    
    @property
    def dispatcher(self) -> Dispatcher:
        """obtain the current registered dispatcher instance of the environment"""
        if self._dispatcher is None:
            raise ValueError("No Dipsatcher instance registered.")
        else:
            return self._dispatcher
    
    def register_resource(
        self,
        obj: InfstructObj,
        custom_identifier: CustomID,
        name: str | None,
    ) -> tuple[EnvID, str]:
        """
        registers an infrastructure object in the environment by assigning an unique id and 
        adding the object to the associated resources of the environment
        
        object:     env resource
        returns:
            env_id: assigned env ID
        """
        # check for uniqueness of custom_identifier
        # type security
        if not isinstance(custom_identifier, (str, int)):
            raise TypeError("Custom identifier must be of type STR or INT")
        # create check value
        if isinstance(custom_identifier, str):
            # remove capital letters for checking
            check_val = custom_identifier.lower()
        else:
            check_val = custom_identifier
        
        # check if value already exists
        if check_val in self._res_custom_identifiers:
            raise ValueError(f"The custom identifier {custom_identifier} provided already exists, \
                            but has to be unique.")
        else:
            self._res_custom_identifiers.add(check_val)
        
        # obtain env_id
        env_id = self._obtain_env_id()
        
        # custom name
        if name is None:
            name = f'M_env_{env_id}'
        
        # new entry for resource data base
        new_entry: DataFrame = pd.DataFrame({
                                'env_id': [env_id],
                                'custom_id': [custom_identifier],
                                'resource': [obj],
                                'name': [name],
                                'res_type': [obj.res_type]})
        new_entry: DataFrame = new_entry.astype(self._infstruct_prop)
        self._res_db = pd.concat([self._res_db, new_entry], ignore_index=True)
        
        logger.info(f"Successfully registered object with EnvID {env_id} and name {name}")
        
        return env_id, name
    
    @property
    def res_db(self) -> DataFrame:
        """obtain a current overview of registered objects in the environment"""
        return self._res_db

    #@lru_cache(maxsize=200)
    def get_res_obj_by_prop(
        self,
        property: str, 
        val: EnvID | CustomID | str,
        target_prop: str = 'resource',
    ) -> InfstructObj:
        """
        obtain a resource object from the environment by its property and corresponding value
        properties: env_id, custom_id, name
        """
        # check if property is a filter criterion
        if property not in self._res_lookup_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._res_lookup_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        # filter resource database for prop-value pair
        temp1: Series = self._res_db.loc[self._res_db[property] == val, target_prop]
        # check for empty search result, at least one result necessary
        if len(temp1) == 0:
            raise IndexError(f"There were no resources found for the property '{property}' \
                            with the value '{val}'")
        # check for multiple entries with same prop-value pair
        ########### PERHAPS CHANGE NECESSARY
        ### multiple entries but only one returned --> prone to errors
        elif len(temp1) > 1:
            # warn user
            logger.warning(f"CAUTION: There are multiple resources which share the \
                        same value '{val}' for the property '{property}'. \
                        Only the first entry is returned.")
        
        return temp1.iat[0]
    
        

In [24]:
class Machine(simpy.resources.resource.Resource):
    
    def __init__(
        self,
        env: SimulationEnvironment,
        custom_identifier: CustomID,
        name: str | None = None,
        num_slots: int = 1,
    ) -> None:
        """
        env:        SimPy Environment in which machine is embedded
        num_slots:  capacity of the machine, if multiple processing 
                    slots available at the same time > 1, default=1
        """
        # intialize base class
        super().__init__(env=env, capacity=num_slots)
        
        ############# custom identifiers only over env_id
        ### associate env_id with custom_id in env
        ### lookup env_id of object in environment and obtain custom_id
        
        
        ############# CHECK IF NECESSARY IN FUTURE
        """
        if custom_identifier is not None:
            ret = env.register_custom_identifier(
                    env_ID=self.env_id, custom_identifier=custom_identifier)
        """
        
        
        # assert machine information and register object in the environment
        self._env = env
        self.res_type: str = 'Machine'
        self._env_id, self.name = self._env.register_resource(
                                obj=self, custom_identifier=custom_identifier,
                                name=name)
        self.custom_identifier = custom_identifier
        
        # currently processed job
        self.current_job_ID: int | None = None
        self.current_job: Job | None = None
        
        # machine state parameters
        self.is_occupied: bool = False
        self.is_waiting: bool = False
        self.is_blocked: bool = False
        self.is_failed: bool = False
        # maybe for future, curently no working time calendars planned
        self.is_paused: bool = False
        
        # time in state parameters
        self.time_occupied: float = 0.
        self.time_waiting: float = 0.
        self.time_blocked: float = 0.
        self.time_failed: float = 0.
        
        # number of inputs/outputs
        self.num_jobs_input: int = 0
        self.num_jobs_output: int = 0
    
    @property
    def env_id(self) -> EnvID:
        return self._env_id
    
    @property
    def env(self) -> SimulationEnvironment:
        return self._env

In [25]:
(n_jobs, n_machines, n_ops, 
 mat_ProcTimes, mat_JobMachID, mat_OpID) = gen_rnd_JSSP_inst(2,3)

In [26]:
mat_ProcTimes

array([[2, 1, 9],
       [7, 9, 6]], dtype=uint16)

In [27]:
mat_JobMachID

array([[2, 0, 1],
       [0, 1, 2]], dtype=uint16)

In [28]:
mat_OpID

array([[0, 1, 2],
       [3, 4, 5]], dtype=uint16)

In [29]:
env = SimulationEnvironment()

for machine in np.unique(mat_JobMachID):
    MachInst = Machine(env=env, custom_identifier=machine.item())

INFO:base:Successfully registered object with EnvID 0 and name M_env_0
INFO:base:Successfully registered object with EnvID 1 and name M_env_1
INFO:base:Successfully registered object with EnvID 2 and name M_env_2


In [30]:
machTest = Machine(env=env, custom_identifier='test')

INFO:base:Successfully registered object with EnvID 3 and name M_env_3


In [31]:
res_db = env.res_db
res_db

,env_id,custom_id,resource,name,res_type
0,0,0,<__main__.Machine object at 0x00000258A6D9AAD0>,M_env_0,Machine
1,1,1,<__main__.Machine object at 0x00000258A88E3650>,M_env_1,Machine
2,2,2,<__main__.Machine object at 0x00000258A882CC10>,M_env_2,Machine
3,3,test,<__main__.Machine object at 0x00000258A8686590>,M_env_3,Machine


In [32]:
ret = env.get_res_obj_by_prop(property='env_id', val=2)

In [33]:
ret

- job sets as own class currently not necessary, use standard OrderedDict instead

In [34]:
class JobSet(object):
    
    def __init__(
        self,
        mat_ProcTimes: npt.NDArray[np.uint16],
        mat_JobMachID: npt.NDArray[np.uint16],
        mat_OpID: npt.NDArray[np.uint16],
    ):
        """
        mat_ProcTimes: matrix of processing times | shape=(n_jobs,n_machines)
        mat_JobMachID: matrix of machine IDs per job starting by index 1 | shape=(n_jobs,n_machines)
        mat_OpID: matrix of operation IDs starting by index 1 | shape=(n_jobs,n_machines)
        """
        
        self._jobs = OrderedDict()
        
        for job_id in range(len(mat_ProcTimes)):
            temp1 = mat_ProcTimes[job_id].tolist()
            temp2 = mat_JobMachID[job_id].tolist()
            temp3 = mat_OpID[job_id].tolist()
            job = Job(
                identifier=job_id,
                proc_times=temp1,
                machine_order=temp2,
                operation_identifiers=temp3,
            )
            self._jobs[job_id] = job
            
    def __getitem__(
        self,
        job_id: int,
    ):
        return self._jobs[job_id]
    

---

#### **Adding database approach also to the job dispatcher**
**properties**:
- job_id
- custom_id
- job instance
- name
- product_type (for later implementation of different product types)

#### To-Do:
- [x] ~~register job object in dispatcher~~
- [x] ~~register operations in dispatcher~~
- [ ] self-marking as disposable by jobs
    - includes demarking
    - currently only addition implemented
- [x] ~~add uniqueness check for custom IDs in resource objects (only interface to user)~~
- [x] ~~operations list of jobs as deque~~
- [ ] add status info to job DB? (waiting, processing, ...) (disposable = waiting?)
    - advantage: 
        - one central DB with all information, no cluttered information
        - simple filtering for jobs by current status incl. disposable jobs
- [ ] tracking disjunctive graph model

In [35]:
class Dispatcher(object):
    Dispatcher: TypeAlias = 'Dispatcher'
    Job: TypeAlias = 'Job'
    Operation: TypeAlias = 'Operation'
    
    def __init__(
        self,
        env: SimulationEnvironment,
    ) -> None:
        """
        Dispatcher class for given environment (only one dispatcher for each environment)
        - different functions to monitor all jobs in the environment
        - jobs report back their states to the dispatcher
        """
                
        # job data base as simple Pandas DataFrame
        self._job_prop: dict[str, type] = {
            'job_id': int,
            'custom_id': object,
            'job': object,
            'name': str,
            'job_type': str,
            'status': str,
        }
        self._job_db: DataFrame = pd.DataFrame(columns=list(self._job_prop.keys()))
        self._job_db: DataFrame = self._job_db.astype(self._job_prop)
        self._job_lookup_props: set[str] = set(['job_id', 'custom_id', 'name'])
        
        # operation data base as simple Pandas DataFrame
        self._op_prop: dict[str, type] = {
            'op_id': int,
            'job_id': int,
            'custom_id': object,
            'op': object,
            'name': str,
            'machine': object,
            'status': str,
        }
        self._op_db: DataFrame = pd.DataFrame(columns=list(self._op_prop.keys()))
        self._op_db: DataFrame = self._op_db.astype(self._op_prop)
        self._op_lookup_props: set[str] = set(['op_id', 'job_id', 'custom_id', 'name', 'machine'])
                
        # register in environment and get EnvID
        self._env = env
        self._env_id: EnvID = self._env.register_dispatcher(self)
        
        self._disposable_jobs: dict[int, Job] = dict()
        self.job_pool: OrderedDict[JobID, Job] = OrderedDict()
        # managing IDs
        self._id_types = set(['job', 'op'])
        self._job_id_counter: JobID = 0
        self._op_id_counter: OpID = 0
    
    @property
    def env_id(self) -> EnvID:
        return self._env_id
    
    @property
    def env(self) -> SimulationEnvironment:
        return self._env
    
    def _obtain_job_id(self) -> JobID:
        """Simple counter function for managing job IDs"""
        # assign id and set counter up
        job_id = self._id_counter
        self._id_counter += 1
        
        return job_id
    
    def _obtain_op_id(self) -> OpID:
        """Simple counter function for managing operation IDs"""
        # assign id and set counter up
        op_id = self._op_id_counter
        self._op_id_counter += 1
        
        return op_id
    
    def _obtain_load_obj_id(
        self,
        load_type: str,
    ) -> JobID | OpID:
        """Simple counter function for managing operation IDs"""
        # assign id and set counter up
        
        if load_type not in self._id_types:
            raise ValueError(f"Given type {type} not valid. Choose from '{self._id_types}'")
        
        match load_type:
            case 'job':
                ident_no = self._job_id_counter
                self._job_id_counter += 1
            case 'op':
                ident_no = self._op_id_counter
                self._op_id_counter += 1
        
        return ident_no
    
    ###################### common register function for load objects of different kind
    ### perhaps not best way because complexity increases and readability suffers
    def register_load_obj(
        self,
        load_type: str,
        obj: Job,
        custom_identifier: CustomID,
        name: str | None,
    ) -> tuple[JobID, str]:
        """
        registers an load object in the dispatcher instance by assigning an unique id and 
        adding the object to the associated DB of the dispatcher instance
        allowed types: 'job', 'op'
        """
        # check if load type is allowed
        if load_type not in self._id_types:
            raise ValueError(f"Given type {type} not valid. Choose from '{self._id_types}'")
        
        # obtain id
        match load_type:
            case 'job':
                ident_no = self._obtain_load_obj_id(load_type=load_type)
                # custom name
                if name is None:
                    name = f'J_gen_{job_id}'
                # new entry for job data base
                new_entry: DataFrame = pd.DataFrame({
                                        'job_id': [ident_no],
                                        'custom_id': [custom_identifier],
                                        'job': [obj],
                                        'name': [name],
                                        'job_type': [obj.job_type]})
                new_entry: DataFrame = new_entry.astype(self._job_prop)
                self._job_db = pd.concat([self._job_db, new_entry], ignore_index=True)
                
            case 'op':
                ident_no = self._obtain_load_obj_id(load_type=load_type)
                # custom name
                if name is None:
                    name = f'J_gen_{job_id}'
                # new entry for job data base
                new_entry: DataFrame = pd.DataFrame({
                                        'job_id': [ident_no],
                                        'custom_id': [custom_identifier],
                                        'job': [obj],
                                        'name': [name],
                                        'job_type': [obj.job_type]})
                new_entry: DataFrame = new_entry.astype(self._job_prop)
                self._job_db = pd.concat([self._job_db, new_entry], ignore_index=True)
        
        logger.info(f"Successfully registered load object of type {load_type} with ID {ident_no} and name {name}")
        
        return False
    
    def register_job(
        self,
        obj: Job,
        custom_identifier: CustomID | None,
        name: str | None,
        status: str | None,
    ) -> tuple[JobID, str]:
        """
        registers an job object in the dispatcher instance by assigning an unique id and 
        adding the object to the associated jobs
        
        object:     env resource
        returns:
            env_id: assigned env ID
        """
        # obtain id
        job_id = self._obtain_load_obj_id(load_type='job')
        
        # custom name
        if name is None:
            name = f'J_gen_{job_id}'
        
        # new entry for job data base
        new_entry: DataFrame = pd.DataFrame({
                                'job_id': [job_id],
                                'custom_id': [custom_identifier],
                                'job': [obj],
                                'name': [name],
                                'job_type': [obj.job_type],
                                'status': status})
        new_entry: DataFrame = new_entry.astype(self._job_prop)
        self._job_db = pd.concat([self._job_db, new_entry], ignore_index=True)
        
        logger.info(f"Successfully registered job with JobID {job_id} and name {name}")
        
        return job_id, name
    ################################################################
    ### add OP database
    def register_operation(
        self,
        obj: Operation,
        machine_identifier: MachineID,
        custom_identifier: CustomID | None,
        name: str | None,
        status: str | None, ###### add status types later
    ) -> tuple[OpID, str, Machine]: ##### add machine type later
        """
        registers an job object in the dispatcher instance by assigning an unique id and 
        adding the object to the associated operations
        
        obj: operation to register
        machine_identifier: custom ID of the associated machine (user interface)
        custom_identifier: custom identifier of the operation 
            (kept for consistency reasons, perhaps remove later)
        name: assigned name the operation
        status: for future features if status of operations is tracked
        
        outputs:
        op_id: assigned operation ID
        name: assigned name
        machine: corresponding machine infrastructure object
        """
        # obtain id
        op_id = self._obtain_load_obj_id(load_type='op')
        
        # custom name
        if name is None:
            name = f'O_gen_{op_id}'
        
        # corresponding machine object on which operation is performed
        machine = self._env.get_res_obj_by_prop(property='custom_id', val=machine_identifier)
        
        # new entry for operation data base
        new_entry: DataFrame = pd.DataFrame({
                                'op_id': [op_id],
                                'job_id': [obj.job_id],
                                'custom_id': [custom_identifier],
                                'op': [obj],
                                'name': [name],
                                'machine': [machine],
                                'status': status})
        new_entry: DataFrame = new_entry.astype(self._op_prop)
        self._op_db = pd.concat([self._op_db, new_entry], ignore_index=True)
        
        logger.info(f"Successfully registered operation with OpID {op_id} and name {name}")
        
        ################# return machine object
        return op_id, name, machine
    
    def gen_job_pool_generic(
        self,
        mat_ProcTimes: npt.NDArray[np.uint16],
        mat_JobMachID: npt.NDArray[np.uint16],
        mat_OpID: npt.NDArray[np.uint16],
    ) -> OrderedDict[JobID, Job]:
        """
        function to build a integrated job pool if generic JxM JSSP instances are used
        mat_ProcTimes: matrix of processing times | shape=(n_jobs,n_machines)
        mat_JobMachID: matrix of machine IDs per job starting by index 1 | shape=(n_jobs,n_machines)
        mat_OpID: matrix of operation IDs starting by index 1 | shape=(n_jobs,n_machines)
        """
            
        for job_id in range(len(mat_ProcTimes)):
            temp1 = mat_ProcTimes[job_id].tolist()
            temp2 = mat_JobMachID[job_id].tolist()
            temp3 = mat_OpID[job_id].tolist()
            JobInst = Job(
                dispatcher=self,
                proc_times=temp1,
                machine_order=temp2,
                operation_identifiers=temp3,
                custom_identifier=None,
                name=None,
            )
            ######### ADD TO JOB (bottom-up approach) #######################
            ### jobs add themselves to job pool
            self.job_pool[job_id] = JobInst
            
        return self.job_pool
     
    @property
    def job_db(self) -> DataFrame:
        """
        obtain a current overview of registered jobs in the environment
        """
        return self._job_db
    
    @property
    def op_db(self) -> DataFrame:
        """
        obtain a current overview of registered operations in the environment
        """
        return self._op_db

    #@lru_cache(maxsize=200)
    def get_job_obj_by_prop(
        self,
        property: str, 
        val: EnvID | CustomID | str,
        target_prop: str = 'job',
    ) -> Job:
        """
        obtain a job object from the dispatcher by its property and corresponding value
        properties: job_id, custom_id, name
        """
        # check if property is a filter criterion
        if property not in self._job_lookup_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._job_lookup_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        # filter resource database for prop-value pair
        temp1: Series = self._job_db.loc[self._job_db[property] == val, target_prop]
        # check for empty search result, at least one result necessary
        if len(temp1) == 0:
            raise IndexError(f"There were no resources found for the property '{property}' \
                            with the value '{val}'")
        # check for multiple entries with same prop-value pair
        ########### PERHAPS CHANGE NECESSARY
        ### multiple entries but only one returned --> prone to errors
        elif len(temp1) > 1:
            # warn user
            logger.warning(f"CAUTION: There are multiple resources which share the \
                        same value '{val}' for the property '{property}'. \
                        Only the first entry is returned.")
        
        return temp1.iat[0]
    
    def add_disposable_job(
        self,
        job: Job,
    ) -> None:
        """
        add job to the disposable ones
        """
        self._disposable_jobs[job.job_id] = job
    
    @property
    def disposable_jobs(self) -> dict[int, Job]:
        return self._disposable_jobs
    
    ################# REWORK ##################
    ### maybe add a corresponding property in the job DB
    def get_disposable_jobs(
        self,
        job_set: OrderedDict,
    ) -> tuple[list[JobID], list[Job]]:
        """
        function needs to be reworked, jobs should report back information to a dispatcher instance
        (bottom-up instead of top-down)
        """
        #########################################
        self._disposable_jobs_ID: list[int] = list()
        self._disposable_jobs: list[Job] = list()
        
        for job_id, job in job_set.items():
            if job.is_disposable:
                self._disposable_jobs_ID.append(job_id)
                self._disposable_jobs.append(job)
                
        return self._disposable_jobs_ID, self._disposable_jobs

In [36]:
env = SimulationEnvironment()
for machine in np.unique(mat_JobMachID):
    MachInst = Machine(env=env, custom_identifier=machine.item())
dispatcher = Dispatcher(env)

INFO:base:Successfully registered object with EnvID 0 and name M_env_0
INFO:base:Successfully registered object with EnvID 1 and name M_env_1
INFO:base:Successfully registered object with EnvID 2 and name M_env_2
INFO:base:Successfully registered dispatcher with EnvID 3


In [37]:
ret = env.dispatcher
ret

### Logic behind simulation model approach
*Model of Resource and Load*:
- system consists of physical objects, also called infrastructure
    - each element can be considered as encapsulated resource
- stress can be put on each system by occupying resources, also called load
    - definition of load depends on system type and modelling, e.g. production jobs for production systems or customers for cashiers in a shop
- load objects are called ***load unit***

*Guiding Priciples*:
- **load objects** can only be spatially and temporally modified by **resources**
    - whole **routing logic is implemented in the resources**: no load object can change its state without a associated resource
    - load objects **contain the necessary information** which is essential for their further processing

In [38]:
class Operation(object):
    Job: TypeAlias = 'Job'
    Dispatcher: TypeAlias = 'Dispatcher'
    
    def __init__(
        self,
        dispatcher: Dispatcher,
        job: Job,
        proc_time: float,
        machine_identifier: MachineID,
        custom_identifier: CustomID | None = None,
        name: str | None = None,
        status: str | None = None,
    ) -> None:
        """
        identifier: operation's ID
        proc_times: operation's processing times
        machine_identifier: ID of machine on which operation is processed
        """
        # !!!!!!!!! perhaps processing times in future multiple entries depending on associated machine
        # change of input format necessary, currently only one machine for each operation
        # no groups, no differing processing times for different machines 

        # assert operation information
        self._dispatcher = dispatcher
        self._job = job
        self._job_id = job.job_id
                
        # from dispatcher: op_id, name, target_machine
        self._op_id, self.name, self.target_machine = self.dispatcher.register_operation(
                                                        obj=self, machine_identifier=machine_identifier,
                                                        custom_identifier=custom_identifier, name=name, 
                                                        status=status)
            
        # process information
        self.proc_time = proc_time
        ########### adding machine instances
        ### perhaps adding machine sets if multiple machines possible (machine groups)
        ### assignment of machine instance by dispatcher
        #self.target_machine = machine_identifier
    
    @property   
    def dispatcher(self) -> Dispatcher:
        return self._dispatcher
    
    @property
    def op_id(self) -> OpID:
        return self._op_id
    
    @property
    def job(self) -> Job:
        return self._job
    
    @property
    def job_id(self) -> JobID:
        return self._job_id

Ideas:
- change operation list to deque

In [39]:
class Job(object):
    Job: TypeAlias = 'Job'
    Dispatcher: TypeAlias = 'Dispatcher'
    
    def __init__(
        self,
        dispatcher: Dispatcher,
        proc_times: list[float],
        machine_order: list[int],
        operation_identifiers: list[int],
        custom_identifier: CustomID | None = None,
        name: str | None = None,
        status: str | None = None,
    ) -> None:
        """
        identifier:             job's ID
        proc_times:             list of processing times for each operation
        machine_order:          list of machine IDs
        operation_identifiers:  list of operation IDs
        """
        # intialize base class
        super().__init__()
        
        ### BASIC INFORMATION ###
        # assert job information
        self.custom_identifier = custom_identifier
        self.job_type: str = 'Job'
        self._dispatcher = dispatcher
        
        ### register job instance
        self._job_id, self.name = self.dispatcher.register_job(
                                    obj=self, custom_identifier=self.custom_identifier,
                                    name=name, status=status)
        
        ### OPERATIONS ##
        self.operations = deque()
        
        for idx, op_proc_time in enumerate(proc_times):
            Op = Operation(
                dispatcher=self.dispatcher,
                job=self,
                proc_time=op_proc_time,
                machine_identifier=machine_order[idx],
            )
            self.operations.append(Op)
            
        self.open_operations = self.operations.copy()
        self.total_num_ops: int = len(self.operations)
        self.num_finished_ops: int = 0
        self._current_op: Operation = self.operations[0]
        
        ### STATE ###
        # intra-process job state parameters
        # job is being processed, maybe better naming in future
        self.is_occupied: bool = False
        # waiting state only when released
        self.is_waiting: bool = False
        # if lying on failed machine
        self.is_failed: bool = False
        
        # intra-process time characteristics
        self.time_occupied: float = 0.
        self.time_waiting: float = 0.
        self.time_failed: float = 0.
        
        # inter-process job state parameters
        # first operation scheduled --> released job
        self.is_released: bool = False
        # job's next operation is disposable
        # true for each new job, maybe reworked in future for jobs with
        # a start date later than creation date
        self.is_disposable: bool = True
        # add job to disposable ones
        ret = self.dispatcher.add_disposable_job(self)
        # last operation ended --> finished job
        self.is_finished: bool = False
        
        # inter-process time characteristics
        # time of first operation starting point
        self.time_entry: float = 0.
        # time of last operation ending point
        self.time_exit: float = 0.
        
        # current resource location
        self.current_resource: object | None = None # specify type if class definition finished
    
    @property
    def dispatcher(self) -> Dispatcher:
        return self._dispatcher
    
    @property
    def job_id(self) -> JobID:
        return self._job_id
    
    @property
    def current_op(self) -> Operation:
        """
        returns the current operation of the job
        If a job is currently being processed its current operation is 
        not changed until this operation is finished.
        """
        return self._current_op
    
    def get_next_operation(self) -> Operation:
        """
        get next operation
        """
        op = self.open_operations.popleft()
        self._current_op = op
        
        return op

In [40]:
(n_jobs, n_machines, n_ops, 
 mat_ProcTimes, mat_JobMachID, mat_OpID) = gen_rnd_JSSP_inst(2,3)

In [41]:
mat_ProcTimes

array([[2, 1, 9],
       [7, 9, 6]], dtype=uint16)

In [42]:
mat_JobMachID

array([[2, 0, 1],
       [0, 1, 2]], dtype=uint16)

In [43]:
mat_OpID

array([[0, 1, 2],
       [3, 4, 5]], dtype=uint16)

- for test purposes: use JobMachID as identifier for the corresponding machine
- associate machine index in Job or Operation(?)

In [44]:
env = SimulationEnvironment()
for machine in np.unique(mat_JobMachID):
    MachInst = Machine(env=env, custom_identifier=machine.item())
dispatcher = Dispatcher(env)

INFO:base:Successfully registered object with EnvID 0 and name M_env_0
INFO:base:Successfully registered object with EnvID 1 and name M_env_1
INFO:base:Successfully registered object with EnvID 2 and name M_env_2
INFO:base:Successfully registered dispatcher with EnvID 3


In [45]:
env.res_db

,env_id,custom_id,resource,name,res_type
0,0,0,<__main__.Machine object at 0x00000258A89009D0>,M_env_0,Machine
1,1,1,<__main__.Machine object at 0x00000258A88E1450>,M_env_1,Machine
2,2,2,<__main__.Machine object at 0x00000258A88F5F90>,M_env_2,Machine


In [46]:
mach = env.get_res_obj_by_prop(property='custom_id', val=0)
mach

In [47]:
job_pool = dispatcher.gen_job_pool_generic(
    mat_ProcTimes=mat_ProcTimes,
    mat_JobMachID=mat_JobMachID,
    mat_OpID=mat_OpID,
)

INFO:base:Successfully registered job with JobID 0 and name J_gen_0
INFO:base:Successfully registered operation with OpID 0 and name O_gen_0
INFO:base:Successfully registered operation with OpID 1 and name O_gen_1
INFO:base:Successfully registered operation with OpID 2 and name O_gen_2
INFO:base:Successfully registered job with JobID 1 and name J_gen_1
INFO:base:Successfully registered operation with OpID 3 and name O_gen_3
INFO:base:Successfully registered operation with OpID 4 and name O_gen_4
INFO:base:Successfully registered operation with OpID 5 and name O_gen_5


In [48]:
ret = dispatcher.job_db
ret

,job_id,custom_id,job,name,job_type,status
0,0,None,<__main__.Job object at 0x00000258A892F610>,J_gen_0,Job,None
1,1,None,<__main__.Job object at 0x00000258A831C510>,J_gen_1,Job,None


In [49]:
env.res_db

,env_id,custom_id,resource,name,res_type
0,0,0,<__main__.Machine object at 0x00000258A89009D0>,M_env_0,Machine
1,1,1,<__main__.Machine object at 0x00000258A88E1450>,M_env_1,Machine
2,2,2,<__main__.Machine object at 0x00000258A88F5F90>,M_env_2,Machine


In [50]:
mat_JobMachID

array([[2, 0, 1],
       [0, 1, 2]], dtype=uint16)

In [51]:
dispatcher.op_db

,op_id,job_id,custom_id,op,name,machine,status
0,0,0,None,<__main__.Operation object at 0x00000258A86D0D90>,O_gen_0,<__main__.Machine object at 0x00000258A88F5F90>,None
1,1,0,None,<__main__.Operation object at 0x00000258A69B41D0>,O_gen_1,<__main__.Machine object at 0x00000258A89009D0>,None
2,2,0,None,<__main__.Operation object at 0x00000258A51DB350>,O_gen_2,<__main__.Machine object at 0x00000258A88E1450>,None
3,3,1,None,<__main__.Operation object at 0x00000258A8686350>,O_gen_3,<__main__.Machine object at 0x00000258A89009D0>,None
4,4,1,None,<__main__.Operation object at 0x00000258A50B9690>,O_gen_4,<__main__.Machine object at 0x00000258A88E1450>,None
5,5,1,None,<__main__.Operation object at 0x00000258A8813F90>,O_gen_5,<__main__.Machine object at 0x00000258A88F5F90>,None


In [52]:
job = dispatcher.get_job_obj_by_prop(property='job_id', val=0)

In [58]:
job.get_next_operation()

In [56]:
job.current_op

In [54]:
job.operations

deque([<__main__.Operation at 0x16f6b95a450>,
       <__main__.Operation at 0x16f6b7fffd0>])

In [55]:
job.open_operations

deque([<__main__.Operation at 0x16f6b95a450>,
       <__main__.Operation at 0x16f6b7fffd0>])

In [56]:
ret = job.get_next_operation()
ret

In [57]:
job.current_op

In [58]:
job.is_failed

False

In [59]:
op1 = job.operations[0]

In [60]:
op1.dispatcher

In [61]:
op1.target_machine

In [62]:
job_obj = dispatcher.get_job_obj_by_prop(property='job_id', val=1)

In [63]:
job_obj.current_op

In [64]:
env.dispatcher.disposable_jobs

{0: <__main__.Job at 0x16f6ba1d390>, 1: <__main__.Job at 0x16f6b491390>}

---
#### Test simulation logic
- first main goal: execution of a generic example in a simple job shop with single machines and FIFO order

In [6]:
import salabim as sim
from collections import namedtuple

In [7]:
mat_JobMachID

array([[2, 0, 1],
       [0, 1, 2]], dtype=uint16)

In [68]:
env = sim.Environment()
comp = sim.Component(env=env)

In [69]:
comp.env.now()

0

In [8]:
class TestDispatcher():
    def __init__(self) -> None:
        pass
    
    def add_dicts(self, job_dict, machine_dict):
        self.job_dict = job_dict
        self.machine_dict = machine_dict
        
    def get_machine_instance_by_id(self, ident):
        return self.machine_dict[ident]

In [30]:
class TestMachine(sim.Component):
    def __init__(self, machine_id, environment, *args, **kwargs):
        super().__init__(env=environment, *args, **kwargs)
        self.machine_id = machine_id
        self.buffer = sim.Queue(env=environment)
        
    def process(self):
        while True:
            #yield self.passivate
            if len(self.buffer) == 0:
                yield self.passivate()
            print("Machine is getting job from queue")
            # theoretically request to dispatcher possible
            job = self.buffer.pop()
            pro_time = job.pro_time
            print(f"Machine {self} got Job with ID {job.job_id} and ProcTime {pro_time}")
            #print(f"Time is {self.env.now()}")
            # processing
            self.hold(job.pro_time)
            job.activate()
    
        
class TestJob(sim.Component):
    def __init__(self, job_id, ops_proc_times, ops_machine_list, dispatcher, environment, machine_list, *args, **kwargs):
        super().__init__(env=environment, *args, **kwargs)
        self.job_id = job_id
        self.ops_proc_times = ops_proc_times.copy()
        self.ops_machine_list = ops_machine_list.copy()
        self.dispatcher = dispatcher
        self.ops_counter = 0
        self.pro_time = 5
        self.machine_list = machine_list.copy()
        
    def add_dicts(self, machine_dict):
        #self.job_dict = job_dict
        self.machine_dict = machine_dict
        
    def process(self):
        print(f"Job-ID {self.job_id} starts {env.now()}")
        while self.ops_counter < (len(self.ops_proc_times) - 1):
            print("Get Next operation...")
            #self.pro_time = self.ops_proc_times[self.ops_counter]
            machine_id = self.ops_machine_list[self.ops_counter]
            print(f"Operation with ProTime {self.pro_time} on machine {machine_id}")
            print("Obtain machine instance...")
            machine = self.dispatcher.get_machine_instance_by_id(machine_id)
            #machine = self.machine_dict[machine_id]
            #machine = self.machine_list.pop(0)
            print(f"Machine instance is: {machine} and ispassive {machine.ispassive()}")
            
            # now enter buffer
            self.enter(machine.buffer)
            # activate machine if passive
            #if machine.ispassive():
            machine.activate()
            yield self.passivate()
            
            
            self.ops_counter += 1
            print(f"Job-ID {self.job_id} ends {self.env.now()}")

In [31]:
test_dispatcher = TestDispatcher()

In [37]:
test_ProcTimes = mat_ProcTimes.copy()
#test_ProcTimes = np.expand_dims(test_ProcTimes, 0)

test_JobMachID = mat_JobMachID.copy()
#test_JobMachID = np.expand_dims(test_JobMachID, 0)

In [54]:
class TestMachine(sim.Component):
    def __init__(self, machine_id, env, *args, **kwargs):
        super().__init__(env=env, *args, **kwargs)
        self.machine_id = machine_id
        self.buffer = sim.Queue(env=env)
    
    def process(self):
        while True:
            if len(self.buffer) == 0:
                yield self.passivate()
            job = self.buffer.pop()
            proc_time = job.proc_time
            print(f"[START] job ID {job.job_id} at {self.env.now()} on machine ID {self.machine_id} with proc time {job.proc_time}")
            yield self.hold(job.proc_time)
            print(f"[END] job ID {job.job_id} at {self.env.now()} on machine ID {self.machine_id}")
            job.activate()

class TestJob(sim.Component):
    def __init__(self, ident, machine_order, operation_procs, env, dispatcher, *args, **kwargs):
        super().__init__(env=env, *args, **kwargs)
        #self.machine_id = machine_id
        self.machine_list = machine_list.copy()
        self.proc_time = None
        self.counter = 0
        self.operation_procs = operation_procs.copy()
        #self.machine_dict = machine_dict.copy()
        #self.dict_keys = list(machine_dict.keys())
        self.machine_order = machine_order.copy()
        self.job_id = ident
        self.dispatcher = dispatcher
        
    def process(self):
        while len(self.operation_procs) != 0:
            #print(f"Job-ID {self.job_id} starts {env.now()}")
            print(f"Job-ID {self.job_id} \t Operation with ID {self.counter}")
            self.proc_time = self.operation_procs.pop(0)
            #machine = machine_list.pop(0)
            #machine = self.machine_dict[self.dict_keys[self.counter]]
            machine_id = self.machine_order.pop(0)
            machine = self.dispatcher.get_machine_instance_by_id(machine_id)
            #print(f"Machine instance is {machine}")
            #print(f"Machine instance is: {machine} and ispassive {machine.ispassive()}")
            self.enter(machine.buffer)
            if machine.ispassive():
                machine.activate()
            yield self.passivate()
            #print(f"[END] Time now Job-ID {self.job_id} is {self.env.now()}")
            self.counter += 1

In [55]:
env = sim.Environment(trace=False)
machine_db = dict()
job_db = dict()
machine_list_inst = list()

for machine_id in np.unique(mat_JobMachID):
    machine = TestMachine(machine_id=machine_id.item(), env=env)
    machine_db[machine_id.item()] = machine
    machine_list_inst.append(machine)

for job_id, (proc_list, machine_list) in enumerate(zip(test_ProcTimes, test_JobMachID)):
    job = TestJob(ident=job_id, machine_order=machine_list.tolist(), operation_procs=proc_list.tolist(), 
                  dispatcher=test_dispatcher, env=env)
    #job.add_dicts(machine_db)
    job_db[job_id] = job
"""
proc_list = [10,8,6]
machine_list = [0,1,2]
job_id = 0
job = TestJob(ident=job_id, machine_dict=machine_db, operation_procs=proc_list,
                  dispatcher=test_dispatcher, env=env)
"""
   
test_dispatcher.add_dicts(job_db, machine_db)


In [59]:
str(machine)

'TestMachine (testmachine.2)'

In [56]:
env.run()

Job-ID 0 	 Operation with ID 0
Job-ID 1 	 Operation with ID 0
[START] job ID 0 at 0.0 on machine ID 2 with proc time 2
[START] job ID 1 at 0.0 on machine ID 0 with proc time 7
[END] job ID 0 at 2.0 on machine ID 2
Job-ID 0 	 Operation with ID 1
[END] job ID 1 at 7.0 on machine ID 0
[START] job ID 0 at 7.0 on machine ID 0 with proc time 1
Job-ID 1 	 Operation with ID 1
[START] job ID 1 at 7.0 on machine ID 1 with proc time 9
[END] job ID 0 at 8.0 on machine ID 0
Job-ID 0 	 Operation with ID 2
[END] job ID 1 at 16.0 on machine ID 1
[START] job ID 0 at 16.0 on machine ID 1 with proc time 9
Job-ID 1 	 Operation with ID 2
[START] job ID 1 at 16.0 on machine ID 2 with proc time 6
[END] job ID 1 at 22.0 on machine ID 2
[END] job ID 0 at 25.0 on machine ID 1


In [41]:
mat_JobMachID

array([[2, 0, 1],
       [0, 1, 2]], dtype=uint16)

In [42]:
mat_ProcTimes

array([[2, 1, 9],
       [7, 9, 6]], dtype=uint16)

single job routing (without buffer):
- job looks for next operation
- job requests machine instance directly by activating it

In [179]:
# Bank, 1 clerk.py

import salabim as sim



class CustomerGenerator(sim.Component):

    def process(self):

        while True:

            Customer()

            yield self.hold(sim.Uniform(5, 15).sample())



class Customer(sim.Component):

    def process(self):

        self.enter(waitingline)

        if clerk.ispassive():

            clerk.activate()

        yield self.passivate()



class Clerk(sim.Component):

    def process(self):

        while True:

            while len(waitingline) == 0:

                yield self.passivate()

            self.customer = waitingline.pop()

            yield self.hold(30)

            print(f"Test env time {self.env.now()}")
            self.customer.activate()



env = sim.Environment(trace=False)


CustomerGenerator()

clerk = Clerk()

waitingline = sim.Queue("waitingline")


env.run(till=50)

print()

waitingline.print_statistics()

Test env time 30.0

Statistics of waitingline at        50    
                                                                     all    excl.zero         zero
-------------------------------------------- -------------- ------------ ------------ ------------
Length of waitingline                        duration             50           35.369       14.631
                                             mean                  1.410        1.993
                                             std.deviation         1.107        0.754

                                             minimum               0            1    
                                             median                2            2    
                                             90% percentile        3            3    
                                             95% percentile        3            3    
                                             maximum               3            3    

Length of stay in waitingline       

In [67]:
import simpy
env = simpy.Environment()

In [68]:
machine = simpy.Resource(env)

In [69]:
def item_generator(env):
    TIME_FOR_CREATION = 10
    counter = 0
    
    while counter < 15:
        # wait for creation time
        yield env.timeout(TIME_FOR_CREATION)
        # item is generated
        print(f"Item generated at time {env.now}")
        counter += 1
    

In [70]:
env.process(item_generator(env))
env.run()

Item generated at time 10
Item generated at time 20
Item generated at time 30
Item generated at time 40
Item generated at time 50
Item generated at time 60
Item generated at time 70
Item generated at time 80
Item generated at time 90
Item generated at time 100
Item generated at time 110
Item generated at time 120
Item generated at time 130
Item generated at time 140
Item generated at time 150


In [71]:
class Source(simpy.resources.store.Store):
    
    def __init__(
        self,
        env: SimulationEnvironment,
        capacity: float = float('inf')
    ) -> None:
        self._contents = deque()
    
    def input_item(self, item): # _do_put implementation
        self._contents.appendleft(item)
        
    def output_item(self, rule):
        # define rule to choose an item
        # first check FIFO as easiest
        
        # FIFO only popping from the right auf the deque
        job = self._contents.pop()
        
        # now analyze item and trigger placement process
        # look for next operation in the job
        # get next machine instance of that operation
        # try placing job in that machine by requesting in separate process

In [72]:
from simpy.resources.store import Store
from collections import namedtuple

In [73]:
Job = namedtuple('Job', ['id', 'operations'])

In [74]:
job = Job(0, [0,1,2])

In [75]:
source = Store(env=env)

---

In [ ]:
import salabim as sim
import simpy
import random

In [ ]:
random.seed(42)

In [ ]:
def execution_time():
    return random.normalvariate(mu=10., sigma=1.5)

In [ ]:
def test_process_put(env, store):
    TIME = 10
    counter = 10
    while counter:
        item = yield store.get()
        print(f"I got item {item} at {env.now}")
        yield env.timeout(TIME)
        #yield env.timeout(TIME)
        #print(f"I got executed at {env.now}")
        counter -= 1
        
def placer_process(env, store):
    TIME = 2
    counter = 6
    while counter:
        yield env.timeout(TIME)
        yield store.put(counter)
        print(f"Placed object {counter} in store")
        counter -= 1
        
def placer_process_machine(env, machine):
    TIME = 2
    counter = 6
    PROCESSING_TIME = 10
    while counter:
        #yield env.timeout(TIME)
        with machine.request() as req:
            yield req
        yield env.timeout(PROCESSING_TIME)
        #yield machine.put(counter)
        print(f"Item processed on machine {machine} at {env.now}")
        counter -= 1

In [ ]:
env = simpy.Environment()
store = simpy.Store(env=env)
store.put(1)
machine = simpy.resources.resource.Resource(env=env)
#env.process(test_process_put(env=env, store=store))
#env.process(placer_process(env=env, store=store))
env.process(placer_process_machine(env=env, machine=machine))


<Process(placer_process_machine) object at 0x2203c09eb10>

In [ ]:
env.run()

Item processed on machine <simpy.resources.resource.Resource object at 0x000002203C149E50> at 10
Item processed on machine <simpy.resources.resource.Resource object at 0x000002203C149E50> at 20
Item processed on machine <simpy.resources.resource.Resource object at 0x000002203C149E50> at 30
Item processed on machine <simpy.resources.resource.Resource object at 0x000002203C149E50> at 40
Item processed on machine <simpy.resources.resource.Resource object at 0x000002203C149E50> at 50
Item processed on machine <simpy.resources.resource.Resource object at 0x000002203C149E50> at 60
